In [18]:
import sys
sys.path.append("../src")
from bigbrotr import Bigbrotr
from event import Event
from relay import Relay
from relay_metadata import RelayMetadata
import utils
import time

# Utils

In [ ]:
private_key, public_key = utils.generate_nostr_keypair()
e = utils.generate_event(private_key, public_key, 1, [], "test")
assert utils.verify_sig(e['id'], e['pubkey'], e['sig']), "Signature verification failed"
assert utils.calc_event_id(e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content']) == e['id'], "Event ID calculation failed"
e = Event(e['id'], e['pubkey'], e['created_at'], e['kind'], e['tags'], e['content'], e['sig'])
e

# Database

In [19]:
bigbrotr = Bigbrotr(
    host="localhost",
    port=5432,
    user="admin",
    password="admin",
    dbname="bigbrotr"
)

In [20]:
bigbrotr.connect()

In [21]:
query = "SELECT * FROM relay_metadata"
bigbrotr.execute(query)
rows = bigbrotr.fetchall()
for row in rows:
    print(row)

In [ ]:
bigbrotr.close()

# relay_urls

In [ ]:
import pandas as pd
relays_url = pd.read_csv("../seed/relays_url.csv")
relays_url = relays_url.groupby('relay_url').agg({'count': 'sum'}).reset_index()
relays_url = relays_url.sort_values(by='count', ascending=False).reset_index(drop=True)
relays_url.to_csv("../seed/relays_url.csv", index=False)

# compute_relay_metadata

In [17]:
from relay import Relay
from aiohttp import ClientSession, ClientTimeout

relay = Relay('wss://hilariouschaos.com')

async def fetch_nip11_metadata(relay_id, session, timeout):
    headers = {'Accept': 'application/nostr+json'}
    for schema in ['https://', 'http://']:
        try:
            async with session.get(schema + relay_id, headers=headers, timeout=timeout) as response:
                if response.status == 200:
                    return await response.json()
                else:
                    pass
        except Exception as e:
            pass
    return None

relay_id = relay.url.removeprefix('wss://')
nip11_raw = None
async with ClientSession() as session:
    nip11_raw = await fetch_nip11_metadata(relay_id, session, 10)
nip11_raw

{'@context': ['https://join-lemmy.org/context.json',
  'https://www.w3.org/ns/activitystreams'],
 'type': 'Application',
 'id': 'https://hilariouschaos.com/',
 'name': 'Hilarious Chaos',
 'preferredUsername': 'hilariouschaos.com',
 'inbox': 'https://hilariouschaos.com/inbox',
 'outbox': 'https://hilariouschaos.com/site_outbox',
 'publicKey': {'id': 'https://hilariouschaos.com/#main-key',
  'owner': 'https://hilariouschaos.com/',
  'publicKeyPem': '-----BEGIN PUBLIC KEY-----\nMIIBIjANBgkqhkiG9w0BAQEFAAOCAQ8AMIIBCgKCAQEAtoxiwzOgNEclE3LSxkFB\nYpNOD1/FJu891peiuBMZWuf3SjRqmAFXXE2zKExhCXolPdmLpdRrRMsuDY+rc8ZI\nRrF+yK/WGkb8ZxWnSUuy6lwVt3V+dawQ1QL0G3t+O5lfumeK3fmwvVWwaSgExLsL\nVbeB5ULGTK6ZOmUNe+N5s7+KMpehfGb1l17oxOkyQVAVKt3Gc0shn1fDiyiFZjCz\n+hMhBOGljPyUQsCzFKpgpaIJ2D7fbwp5sfZ642FDfsvIfUsChMd5Zn4SDijBy3Xg\nc8T3uY2siNTmdbg+PdsIdsiV12U0aOGdrkVDYwFQRlVoeglISiEOuSkugD8jzNc9\n6QIDAQAB\n-----END PUBLIC KEY-----\n'},
 'content': '<p><strong>Welcome to Hilarious Chaos!</strong></p>\n<p>You are welcome